In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
#         print(os.path.join(dirname, filename))
#
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [14]:
from pathlib import Path
import shutil
from bs4 import BeautifulSoup
import os

In [15]:
def normalized_coordinates(filename, width, height, xmin, ymin, xmax, ymax):
    """Take in image coordinates (unnormalized) as input, return normalized values 
    """
    
    xmin, xmax = xmin / width, xmax / width
    ymin, ymax = ymin / height, ymax/ height

    width = xmax-xmin
    height = ymax-ymin
    x_center = xmin + (width / 2)
    y_center = ymin + (height / 2)

    return x_center, y_center, width, height

def write_label(filename, x_center, y_center, width, height):
    """Save image's coordinates in text file named "filename"
    """
    with open(filename, mode='w') as outf:
        outf.write(f"{0} {x_center} {y_center} {width} {height}\n")
        
def parse_xml_tags(data):
    """Parse xml label file, return image file name, and its coordinates as a dictionary
    """
    tags = ['filename', 'width', 'height', 'xmin', 'ymin', 'xmax', 'ymax']
    Bs_data = BeautifulSoup(data, "xml")
    d = dict()

    for t in tags:
        text = Bs_data.find(t).text
        if all(c.isdigit() for c in text):
            d[t] = int(text)
        else:
            d[t] = text
    return d

def build_data(dir_folder, ann_file_list, img_dir):
    """Write xml labels to text file with specifications format, save at 'labels' folder.
        Move image to 'images' folder
    """
    images_folder = f"{dir_folder}/images"
    labels_folder = f"{dir_folder}/labels"
    
    os.makedirs(images_folder, exist_ok = True)
    os.makedirs(labels_folder, exist_ok = True)


    for ann_file in ann_file_list:
        with open(ann_file, 'r') as f:
            label = parse_xml_tags(f.read())
         
        img_file_name = label['filename']
        x_center, y_center, width, height = normalized_coordinates(**label)
         
        # save at 'labels' folder
        write_label(f"{labels_folder}/{img_file_name.split('.')[0]}.txt", x_center, y_center, width, height)
         
         # Move image to 'images' folder
        shutil.copy(f"{img_dir}/{img_file_name}", f"{images_folder}/{img_file_name}")

In [16]:
import glob

dir_folder = "plate_datasets"

ann_list = glob.glob('/kaggle/input/number-plate-detection/images/*.xml')
build_data('datasets/'+dir_folder, ann_list, "/kaggle/input/number-plate-detection/images")

In [17]:
import yaml

data = {
    "path": dir_folder,
    "train": "images",
    "val": "images",
    "names": {0: "car_lisence_plate"}
    }

with open('dataset.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

In [18]:
assert len(os.listdir("/kaggle/working/datasets/plate_datasets/labels")) == len(os.listdir("/kaggle/working/datasets/plate_datasets/images"))

In [19]:
import torch
torch.cuda.is_available()

True

In [20]:
!pip install ultralytics

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [21]:
from ultralytics import YOLO 

model = YOLO("yolov8m.pt")


In [22]:
results = model.train(data='/kaggle/working/dataset.yaml', epochs=200, batch=40, imgsz=640,  project='runs', name='exp',  exist_ok=True, device='cuda')

Ultralytics YOLOv8.2.77 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/kaggle/working/dataset.yaml, epochs=200, time=None, patience=100, batch=40, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=runs, name=exp, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True,

wandb: Currently logged in as: yuval-mehta57 (llm-yoddha). Use `wandb login --relogin` to force relogin


Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /kaggle/working/datasets/plate_datasets/labels.cache... 225 images, 0 backgrounds, 0 corrupt: 100%|██████████| 225/225 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

                   all        225        225      0.987          1      0.995      0.946



200 epochs completed in 0.720 hours.
Optimizer stripped from runs/exp/weights/last.pt, 52.1MB
Optimizer stripped from runs/exp/weights/best.pt, 52.1MB

Validating runs/exp/weights/best.pt...
Ultralytics YOLOv8.2.77 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 218 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.24s/it]


                   all        225        225      0.987          1      0.995      0.946
Speed: 0.1ms preprocess, 9.9ms inference, 0.0ms loss, 3.7ms postprocess per image
Results saved to runs/exp


lr/pg0,▂▄▆████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg1,▂▄▆████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg2,▂▄▆████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
metrics/mAP50(B),▇▁▁▁▁▃██████████████████████████████████
metrics/mAP50-95(B),▄▁▁▁▁▂▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████████
metrics/precision(B),▇▁▁▁▁▅▇▇████████████████████████████████
metrics/recall(B),▅▂▃▁▂▃▇█▇▇██████████████████████████████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▆▇▇▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁


In [23]:
results= model("/kaggle/input/number-plate-detection/TEST/TEST.jpeg")
# results[0].save()
len(results)


image 1/1 /kaggle/input/number-plate-detection/TEST/TEST.jpeg: 544x640 1 car_lisence_plate, 89.7ms
Speed: 2.5ms preprocess, 89.7ms inference, 1.5ms postprocess per image at shape (1, 3, 544, 640)


1